In [1]:
!pip install streamlit-folium
!pip install geopandas

     |████████████████████████████████| 94 kB 3.3 MB/s 
     |████████████████████████████████| 8.3 MB 30.7 MB/s 
     |████████████████████████████████| 180 kB 45.3 MB/s 
     |████████████████████████████████| 4.3 MB 50.8 MB/s 
     |████████████████████████████████| 76 kB 4.6 MB/s 
     |████████████████████████████████| 111 kB 75.2 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 124 kB 52.2 MB/s 
     |████████████████████████████████| 788 kB 53.2 MB/s 
     |████████████████████████████████| 374 kB 61.8 MB/s 
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13478 sha256=6443f987ce606f15d33ce56cf2f6d5c11463cc2cf33b3fd9467632703b60413d
  Stored in directory: /root/.cache/pip/wheels/22/f5/18/df711b66eb25b21325c132757d4314db9ac5e8dabeaf196eab
Successfully built blinker
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Su

     |████████████████████████████████| 1.0 MB 30.6 MB/s 
     |████████████████████████████████| 15.4 MB 39 kB/s 
     |████████████████████████████████| 6.3 MB 41.6 MB/s 


In [8]:
%%writefile app.py
#version 4

import streamlit as st
import pandas as pd
import folium
from folium.plugins import BeautifyIcon
from streamlit_folium import folium_static
import geopandas as gpd
import shapely
import calendar
import branca.colormap as cm

@st.cache(persist=True, allow_output_mutation=True)
def process_data():
  all_three = pd.read_csv('./temp_pr_spei.csv')
  all_three = (gpd.GeoDataFrame(all_three, crs='epsg:4326', geometry= all_three.apply(lambda x: 
                  shapely.geometry.box(x['lon']-0.25, x['lat']-0.25, x['lon']+0.25, x['lat']+0.25 ), axis=1)))
  
  spei = pd.read_csv('/content/historical_spei_gamma.csv')
  spei_grid = gpd.GeoDataFrame(spei,crs='epsg:4326', geometry=spei.apply(lambda x: shapely.geometry.box(x['lon']-0.25, x['lat']-0.25, x['lon']+0.25, x['lat']+0.25 ), axis=1))
  
  temp = pd.read_csv('/content/historical_tmp.csv')
  temp_grid = gpd.GeoDataFrame(temp,crs='epsg:4326', geometry=temp.apply(lambda x: shapely.geometry.box(x['lon']-0.25, x['lat']-0.25, x['lon']+0.25, x['lat']+0.25 ), axis=1))

  rain = pd.read_csv('/content/historical_pr.csv')
  rain_grid = gpd.GeoDataFrame(rain,crs='epsg:4326', geometry=rain.apply(lambda x: shapely.geometry.box(x['lon']-0.25, x['lat']-0.25, x['lon']+0.25, x['lat']+0.25 ), axis=1))

  return all_three, temp_grid, rain_grid, spei_grid


all_three, temp, rain, spei = process_data()

gdf = all_three.copy()
temp = temp.copy()
rain = rain.copy()
spei = spei.copy()

x = all_three['lon'].median()
y = all_three['lat'].median()


option_dict = {'Temperature':'tmp', 'Rainfall':'pre', 'SPEI':'spei_gamma_03'}

# st.markdown('## Weather and Drought Analysis')

st.write('''## Weather and Drought Analysis


The map below shows the spatial and temporal distribution of Temperature and Rainfall, and also one drought index, 
the Standardized Precipitation Evapotranspiration Index (SPEI).


The SPEI is a multiscalar drought index based on climatic data. It can be used for determining the onset, 
duration and magnitude of drought conditions with respect to normal conditions in a variety of natural and managed 
systems such as crops, ecosystems, rivers, water resources, etc. SPEI is a standardized index. Values is between -1 and -2
are said to mean moderate drought, between -2 and -3 are said to mean severe drought, whereas values lower than -3 are 
said to mean extreme drought.


The figures show that the Northern regions are hotter and drier, while the Southern regions are cooler and wetter.
''')


selected = st.radio('Pick a variable', options=option_dict.keys())

v = option_dict[selected]

if selected == 'Temperature':
  color = 'OrRd'
elif selected =='Rainfall':
  color = 'Blues'
elif selected == 'SPEI':
  color = 'RdYlBu'

mon_annual = st.radio('Select level of aggregation', options = ['Annual', 'Monthly'])

if mon_annual == 'Annual':
  if selected in ['Temperature', 'SPEI']:
    agg_func = 'mean'
  elif selected == 'Rainfall':
    agg_func = 'sum'
  
  temp2 = gdf.groupby(['lat', 'lon'])[[v]].agg(agg_func).merge(gdf[['geometry', 'lat', 'lon']].drop_duplicates(), on=['lat','lon']).reset_index()
  temp2.columns = ['division', 'lat', 'lon', v, 'geometry']
  
  temp2.set_crs(epsg=4326, inplace=True)


  m = folium.Map(location=[y,x], zoom_start=6)


  choropleth = folium.Choropleth(
      geo_data=temp2,
      name="choropleth",
      data=temp2,
      columns=['division',v],
      key_on="feature.properties.division",
      fill_color=color,
      fill_opacity=0.7,
      line_opacity=0.1,
      legend_name=v,
  ).add_to(m)

  choropleth.geojson.add_child(
      folium.features.GeoJsonTooltip([v])
  )

  st.write('Annual distribution of '+ selected)
  folium_static(m)

elif mon_annual=='Monthly':

    month = st.selectbox('Pick a month', calendar.month_name[1:])
    
    m = folium.Map(location=[y,x], zoom_start=6)


    choropleth = folium.Choropleth(
        geo_data=gdf.loc[gdf['Month_names']==month].reset_index()[['division', v,'geometry']],
        name="choropleth",
        data=gdf.loc[gdf['Month_names']==month].reset_index()[['division', v,'geometry']],
        columns=['division',v],
        key_on="feature.properties.division",
        fill_color=color,
        fill_opacity=0.7,
        line_opacity=0.1,
        legend_name=v,
    ).add_to(m)

    choropleth.geojson.add_child(
        folium.features.GeoJsonTooltip([v])
    )


    st.write(selected + ' for the month of '+ month)
    folium_static(m)
    


st.header('Analyzing trends in Temperature, rainfall, and SPEI')

st.write('''The Modified Mann Kendall test was used to assess the trend in rainfall, temperature, and SPEI for different months.
The data was prepared for the test by smoothing it with a 50 year moving average and a 10 year time step.
The grid points are colored by the slope, in case of temperature it is change in degree Celsius per decade, for rainfall it is 
change in mm per decade, and for spei it is change in SPEI per decade. Grid points that have a significant increase 
are marked with a "+" sign, and those that have a significant decrease are marked with a "-" sign.


The figures show that temperature is increasing in most locations, and rainfall is decreasing. The SPEI is also decreasing
 in many locations which means that chances of drought are increasing.''')
st.subheader('Trends in Temperature')

month1 = st.selectbox('Pick a month', calendar.month_name[1:])

m1 = folium.Map(location=[y,x], zoom_start=6)

choropleth = folium.Choropleth(
    geo_data=temp.loc[temp['month']==month1, ['division', 'slope', 'geometry']],
    name="choropleth",
    data=temp.loc[temp['month']==month1,['division', 'slope', 'geometry']],
    columns=['division','slope'],
    key_on="feature.properties.division",
    fill_color='PuOr',
    fill_opacity=0.7,
    line_opacity=0.1,
    legend_name='slope',
).add_to(m1)


choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['slope'])
)

for index, row in temp.loc[(temp['month']==month1 )& temp['h'] & (temp['z'] >0)].iterrows():
  icon_plus = BeautifyIcon(
    icon='plus',
    icon_shape=None, 
    border_color=None, 
    border_width=0,
    background_color='')
  folium.Marker(
        location=[row['lat'], row['lon']], 
        icon=icon_plus).add_to(m1)

for index, row in temp.loc[(temp['month']== month1 ) & temp['h'] & (temp['z'] <0)].iterrows():
  icon_minus = BeautifyIcon(
    icon='minus',
    icon_shape=None, 
    border_color=None, 
    border_width=0,
    background_color='')
  folium.Marker(
        location=[row['lat'], row['lon']], 
        icon=icon_minus).add_to(m1)

folium_static(m1)

st.subheader('Trends in Rainfall')

m2 = folium.Map(location=[y,x], zoom_start=6)

choropleth = folium.Choropleth(
    geo_data=temp.loc[rain['month']==month1, ['division', 'slope', 'geometry']],
    name="choropleth",
    data=temp.loc[rain['month']==month1,['division', 'slope', 'geometry']],
    columns=['division','slope'],
    key_on="feature.properties.division",
    fill_color='PuOr',
    fill_opacity=0.7,
    line_opacity=0.1,
    legend_name='slope',
).add_to(m2)


choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['slope'])
)

for index, row in rain.loc[(rain['month']==month1 )& temp['h'] & (temp['z'] >0)].iterrows():
  icon_plus = BeautifyIcon(
    icon='plus',
    icon_shape=None, 
    border_color=None, 
    border_width=0,
    background_color='')
  folium.Marker(
        location=[row['lat'], row['lon']], 
        icon=icon_plus).add_to(m2)

for index, row in rain.loc[(rain['month']==month1 ) & rain['h'] & (rain['z'] <0)].iterrows():
  icon_minus = BeautifyIcon(
    icon='minus',
    icon_shape=None, 
    border_color=None, 
    border_width=0,
    background_color='')
  folium.Marker(
        location=[row['lat'], row['lon']], 
        icon=icon_minus).add_to(m2)

folium_static(m2)


st.subheader('Trends in SPEI')

m3 = folium.Map(location=[y,x], zoom_start=6)

choropleth = folium.Choropleth(
    geo_data=spei.loc[spei['month']==month1, ['division', 'slope', 'geometry']],
    name="choropleth",
    data=spei.loc[spei['month']==month1,['division', 'slope', 'geometry']],
    columns=['division','slope'],
    key_on="feature.properties.division",
    fill_color='PuOr',
    fill_opacity=0.7,
    line_opacity=0.1,
    legend_name='slope',
).add_to(m3)


choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['slope'])
)

for index, row in spei.loc[(spei['month']==month1 )& spei['h'] & (spei['z'] >0)].iterrows():
  icon_plus = BeautifyIcon(
    icon='plus',
    icon_shape=None, 
    border_color=None, 
    border_width=0,
    background_color='')
  folium.Marker(
        location=[row['lat'], row['lon']], 
        icon=icon_plus).add_to(m3)

for index, row in spei.loc[(spei['month']==month1 ) & spei['h'] & (spei['z'] <0)].iterrows():
  icon_minus = BeautifyIcon(
    icon='minus',
    icon_shape=None, 
    border_color=None, 
    border_width=0,
    background_color='')
  folium.Marker(
        location=[row['lat'], row['lon']], 
        icon=icon_minus).add_to(m3)

folium_static(m3)

Overwriting app.py


In [18]:
!streamlit run app.py


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.185.92.157:8501

  Stopping...


In [3]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

--2021-10-29 17:29:26--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 52.202.168.65, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  76.7MB/s    in 0.2s    

2021-10-29 17:29:26 (76.7 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]



In [4]:
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [9]:
get_ipython().system_raw('./ngrok http 8501 &')

In [10]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    'import sys, json; print("Execute the next cell and the go to the following URL: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'

Execute the next cell and the go to the following URL: https://a5ba-34-86-219-69.ngrok.io


In [11]:
!streamlit run app.py


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.86.219.69:8501

2021-10-29 17:50:32.989 NumExpr defaulting to 2 threads.
  Stopping...
^C
